# Isotropic elastic wave equation implementation on a staggered grid

Some preamble here

In [1]:
import devito as dv

In [2]:
# Create a 3D grid
shape = (51, 51, 51)
extent = (500., 500., 500.)  # this is in metres
grid = dv.Grid(shape=shape, extent=extent)

# Vectorial form

One of the advantages of Devito is the high level symbolic API. This allows for straight forward generalisation of expressions between 2D and 3D. 


Leveraging the symbolic algebra capabilities of Devito enables concise, flexible specifications of equations. In this case, it's desirable to represent the isotropic elastic system of equations in the following form:


\begin{cases}
&\frac{dv}{dt} = \underline{\nabla} . \tau \\
&\frac{d \tau}{dt} = \lambda \text{diag}(\underline{\nabla} . \underline{v}) + \mu (\underline{\nabla} \underline{v} + (\underline{\nabla} \underline{v})^T)
\end{cases}


where in 3D, $\underline{v}$ is a vector valued function:

$\underline{v}(t, x, y, z) = (v_x(t, x, y, z), v_y(t, x, y, z), v_z(t, x, y, z))$

and the stress $\tau$ is a symmetric tensor valued function:


$\tau(t, x, y, z) = \begin{bmatrix}\tau_{xx}(t, x, y, z) & \tau_{xy}(t, x, y, z)  & \tau_{xz}(t, x, y, z) \\\tau_{yx}(t, x, y, z) & \tau_{yy}(t, x, y, z) & \tau_{yz}(t, x, y, z)  \\\tau_{zx}(t, x, y, z) & \tau_{zy}(t, x, y, z) & \tau_{zz}(t, x, y, z)\end{bmatrix}$


We show in the following how to setup the elastic wave-equation form Devito's high-level tensorial types.

As $\underline{v}$ is a vector which varies in time, we represent it as a `VectorTimeFunction`.

In [3]:
so = 4
to = 1

v = dv.VectorTimeFunction(name='v', grid=grid, space_order=so, time_order=to)
v

Vector(v_x(t, x + h_x/2, y, z), v_y(t, x, y + h_y/2, z), v_z(t, x, y, z + h_z/2))

We can see that staggering is applied by default.

There is also a `TensorTimeFunction` to store $\tau$:

In [4]:
τ = dv.TensorTimeFunction(name='tau', grid=grid, space_order=so, time_order=to)
τ

Matrix([
[                tau_xx(t, x, y, z), tau_xy(t, x + h_x/2, y + h_y/2, z), tau_xz(t, x + h_x/2, y, z + h_z/2)],
[tau_xy(t, x + h_x/2, y + h_y/2, z),                 tau_yy(t, x, y, z), tau_yz(t, x, y + h_y/2, z + h_z/2)],
[tau_xz(t, x + h_x/2, y, z + h_z/2), tau_yz(t, x, y + h_y/2, z + h_z/2),                 tau_zz(t, x, y, z)]])

Note that staggering is also applied here by default. Furthermore, this tensor is symmetrical:

In [5]:
τ[0, 1] is τ[1, 0]

True

Let's define our parameters. Whilst these will be constant for now, it is trivial to make them vary in space (as will be shown later). 

In [6]:
vs = 1500.  # s-wave velocity in m/s
vp = 2500.  # p-wave velocity in m/s
ρ = 3000.  # density in kg/m^3

b = 1/ρ
μ = vs**2/ρ
λ = (vp**2/ρ) - 2*μ

With this, we can write our equations in the following way:

Firstly, for the velocity equation: $\frac{dv}{dt} = \underline{\nabla} . \tau$, we formulate it as follows:

In [7]:
pde_v = v.dt - b * dv.div(τ)
pde_v

Vector(-0.000333333333333333*Derivative(tau_xx(t, x, y, z), x) - 0.000333333333333333*Derivative(tau_xy(t, x + h_x/2, y + h_y/2, z), y) - 0.000333333333333333*Derivative(tau_xz(t, x + h_x/2, y, z + h_z/2), z) + Derivative(v_x(t, x + h_x/2, y, z), t), -0.000333333333333333*Derivative(tau_xy(t, x + h_x/2, y + h_y/2, z), x) - 0.000333333333333333*Derivative(tau_yy(t, x, y, z), y) - 0.000333333333333333*Derivative(tau_yz(t, x, y + h_y/2, z + h_z/2), z) + Derivative(v_y(t, x, y + h_y/2, z), t), -0.000333333333333333*Derivative(tau_xz(t, x + h_x/2, y, z + h_z/2), x) - 0.000333333333333333*Derivative(tau_yz(t, x, y + h_y/2, z + h_z/2), y) - 0.000333333333333333*Derivative(tau_zz(t, x, y, z), z) + Derivative(v_z(t, x, y, z + h_z/2), t))

And for the stress equation: $\frac{d \tau}{dt} = \lambda \text{diag}(\underline{\nabla} . \underline{v}) + \mu (\underline{\nabla} \underline{v} + (\underline{\nabla} \underline{v})^T)$:

In [8]:
pde_tau = τ.dt - λ * dv.diag(dv.div(v.forward)) - μ * (dv.grad(v.forward) + dv.grad(v.forward).T)
# pde_tau  # uncomment to print (may go off of page)


In [9]:
# Time update 
u_v = dv.Eq(v.forward, dv.solve(pde_v, v.forward))
u_t = dv.Eq(τ.forward,  dv.solve(pde_tau, τ.forward))

We will need to create a source to inject into this field. For this we will import a `RickerSource` from the examples. Note that this is a thin wrapper around `SparseTimeFunction` with a prefilled time series.

In [10]:
from examples.seismic import TimeAxis, RickerSource

t0, tn = 0., 400.
dt = 0.2*min(grid.spacing)/vs  # Courant number is 0.2
time_range = TimeAxis(start=t0, stop=tn, step=dt)

src = RickerSource(name='src', grid=grid, f0=0.010, time_range=time_range)
src.coordinates.data[:] = 250.

# The source injection term
src_xx = src.inject(field=τ.forward[0, 0], expr=dt*src)
src_yy = src.inject(field=τ.forward[1, 1], expr=dt*src)
src_zz = src.inject(field=τ.forward[2, 2], expr=dt*src)

Now we create a Devito `Operator`. Note the order of arguments supplied to the operator - this corresponds to the order of these updates within the time loop.

In [ ]:
op = dv.Operator([u_v, u_t] + src_xx + src_yy + src_zz)
op(dt=dt)

With this, the wavefield can be rendered to show its 3D structure. We will choose the x normal stress field for this visualisation.

In [ ]:
# import numpy as np

# try:
#     import trame
# except ModuleNotFoundError:
#     ! pip install trame
# try:
#     import pyvista as pv
# except ModuleNotFoundError:
#     ! pip install pyvista
#     import pyvista as pv

# rendergrid = pv.UniformGrid()
# rendergrid.dimensions = np.array(τ[0, 0].data[-1].shape) + 1
# rendergrid.origin = (0., 0., 0.)
# rendergrid.spacing = grid.spacing
# rendergrid.cell_data["x normal stress"] = τ[0, 0].data[-1].flatten(order="F")

# sargs = dict(
#     title_font_size=10,
#     label_font_size=10,
#     n_labels=2,
#     color='k',
#     position_x=0.05,
#     position_y=0.05,
#     height=0.15,
#     width=0.3
    
# )

# vmax = np.amax(τ[0, 0].data[-1])
# y_slices = rendergrid.slice_along_axis(n=5, axis="y")
# p = pv.Plotter(notebook=True, window_size=(600,400))
# p.add_mesh(rendergrid.outline(), color="k")
# p.add_mesh(y_slices, cmap='seismic', opacity=0.8, clim=[-vmax, vmax], scalar_bar_args=sargs)
# p.set_background('white')
# p.show()
# # NOTE: you may have to click on the white box below to make the visualisation appear

In the Devito example notebooks you may have encountered various model and propagator objects used to streamline these notebooks. In practice, we want to make our own versions of these to suit the task at hand. We will outline below one potential strategy to do this for this particular problem.

In [9]:
import numpy as np
import devito as dv
import sympy as sp
import matplotlib.pyplot as plt

class FSDomain(dv.SubDomain):
    name = 'fsdomain'
    
    def __init__(self, nbl):
        super().__init__()
        self._nbl = nbl
        
    def define(self, dimensions):
        return {dimensions[-1]: ('left', self.nbl)}
    
    @property
    def nbl(self):
        """Number of boundary layers"""
        return self._nbl
    
    
class MainDomain(dv.SubDomain):
    name = 'maindomain'
    
    def __init__(self, nbl):
        super().__init__()
        self._nbl = nbl
        
    def define(self, dimensions):
        return {dimensions[-1]: ('middle', self.nbl, 0)}
    
    @property
    def nbl(self):
        """Number of boundary layers"""
        return self._nbl
    
    
class ElasticModel:
    """
    This is a class containing parameters and utilities for
    an isotrophic elastic wave model. 
    
    Parameters
    ----------
    vp : ndarray
        p-wave velocity array
    vs : ndarray
        s-wave velocity array
    ρ : ndarray
        Density
    spacing : tuple
        Grid spacing. Default is 10.
    origin : tuple
        Grid origin. Default is zeros.
    nbl : int
        Number of boundary layers. Default is 10.
    free_surface : bool
        Use a free-surface. Default is False.
    space_order : int
        The order of the spatial discretisation. Default is 2.
    
    Attributes
    ----------
    grid : Grid
        Model grid
    τ : TensorTimeFunction
        Stress tensor
    v : VectorTimeFunction
        Particle velocity 
    vp : Function
        p-wave velocity
    vs : Function
        s-wave velocity
    ρ : Function
        Density
    damp : Function
        Damping mask
    """
    def __init__(self, vp, vs, ρ, **kwargs):
        
        if vp.shape != vs.shape or vp.shape != ρ.shape:
            raise ValueError("Arrays have mismatched shapes")
            
        self._nbl = kwargs.get("nbl", 10)
        self._free_surface = kwargs.get("free_surface", False)

        if self.free_surface:
            fs_domain = FSDomain(self.nbl)
            main_domain = MainDomain(self.nbl)
            subdomains = [fs_domain, main_domain]
        else:
            subdomains = []
        spacing = kwargs.get("spacing", 10.)
        origin = kwargs.get("origin", tuple([0.]*len(vp.shape)))
        
        shape = tuple([2*self.nbl + s for s in vp.shape])
        print(spacing, shape)
        extent = tuple([spacing*sha for sha in shape])
        self._grid = dv.Grid(shape=shape, extent=extent, origin=origin,
                             subdomains=subdomains)
        
        self._space_order = kwargs.get("space_order", 2)
        
        self._initialise_functions()
        self._fill_parameters(vp, vs, ρ)
        
        self._initialise_damping()
        
    def _initialise_functions(self):
        """Set up functions found in the equations"""
        self._τ = dv.TensorTimeFunction(name='τ', grid=self.grid,
                                        space_order=self.space_order,
                                        time_order=1)
        
        self._v = dv.VectorTimeFunction(name='v', grid=self.grid,
                                        space_order=self.space_order,
                                        time_order=1)
        
        self._vp = dv.Function(name='vp', grid=self.grid,
                               space_order=self.space_order)
        
        self._vs = dv.Function(name='vs', grid=self.grid,
                               space_order=self.space_order)

        self._ρ = dv.Function(name='ρ', grid=self.grid,
                              space_order=self.space_order)
        
    def _fill_parameters(self, vp, vs, rho):
        """Fill the data of vp, vs, and rho from the supplied arrays"""
        self.vp.data[:] = np.pad(vp, self.nbl, mode='edge')
        self.vs.data[:] = np.pad(vs, self.nbl, mode='edge')
        self.ρ.data[:] = np.pad(rho, self.nbl, mode='edge')
        
    def _initialise_damping(self):
        """
        Set up the damping mask. Note that this will be a simple sponge
        mask with linear tapers for this case.
        """
        self._damp = dv.Function(name='damp', grid=self.grid,
                                 space_order=self.space_order)
        self.damp.data[:] = 1.
        
        eqs = []
        for dim in self.grid.dimensions:
            eqs.append(dv.Eq(self.damp, self.damp*(dv.Max(dim, self.nbl)/self.nbl)))
            
        op_damp = dv.Operator(eqs, name='init_damp')
        op_damp.apply()
            
        plt.imshow(self.damp.data.T)
        plt.colorbar()
        plt.show()
            
            
            
        
        

    @property
    def nbl(self):
        """Number of boundary layers"""
        return self._nbl
    
    @property
    def free_surface(self):
        """Use a free surface"""
        return self._free_surface
    
    @property
    def grid(self):
        """Finite difference grid """
        return self._grid 
    
    @property
    def space_order(self):
        """Order of the spatial discretisation"""
        return self._space_order
    
    @property
    def τ(self):
        """Stress tensor"""
        return self._τ
    
    @property
    def v(self):
        """Particle Velocity"""
        return self._v

    @property
    def vp(self):
        """P-wave velocity"""
        return self._vp

    @property
    def vs(self):
        """S-wave velocity"""
        return self._vs
    
    @property
    def ρ(self):
        """Density"""
        return self._ρ
    
    @property
    def damp(self):
        """Damping mask"""
        return self._damp
    
    
model = ElasticModel(np.random.rand(10, 10), np.random.rand(10, 10), np.random.rand(10, 10))

10.0 (30, 30)


/var/folders/b1/3bsk_dln7nsg9g6f59jch2nw0000gn/T/devito-jitcache-uid501/f546ab3c83210724c3e058dd89ffeb870c5c9051.c:38:52: error: implicit declaration of function 'MAX' is invalid in C99 [-Werror,-Wimplicit-function-declaration]
      float ssa0 = (1.0F/10.0F)*damp[x + 2][y + 2]*MAX(10, x);
                                                   ^
1 error generated.
FAILED compiler invocation:clang -O3 -g -fPIC -Wall -std=c99 -Wno-unused-result -Wno-unused-variable -ffast-math -shared /var/folders/b1/3bsk_dln7nsg9g6f59jch2nw0000gn/T/devito-jitcache-uid501/f546ab3c83210724c3e058dd89ffeb870c5c9051.c -lm -o /var/folders/b1/3bsk_dln7nsg9g6f59jch2nw0000gn/T/devito-jitcache-uid501/f546ab3c83210724c3e058dd89ffeb870c5c9051.dylib


CompileError: module compilation failed